In [1]:
import tensorflow as tf

In [2]:
# MSE
y = tf.constant([1, 2, 3, 0, 2])
y = tf.one_hot(y, depth=4)
y = tf.cast(y, dtype=tf.float32)

In [3]:
out = tf.random.normal([5, 4])

In [4]:
loss1 = tf.reduce_mean(tf.square(y-out))

In [5]:
loss2 = tf.square(tf.norm(y-out))/(5*4)

In [6]:
# MSE函数得到 => [b]
loss3 = tf.reduce_mean(tf.losses.MSE(y,out))

In [7]:
loss1,loss2,loss3

(<tf.Tensor: shape=(), dtype=float32, numpy=2.2940953>,
 <tf.Tensor: shape=(), dtype=float32, numpy=2.2940953>,
 <tf.Tensor: shape=(), dtype=float32, numpy=2.2940953>)

In [8]:
# 交叉熵
# Entropy = -求和P(i)logP(i)
a = tf.fill([4], 0.25)

In [9]:
# tf的log默认以e为底,但是此处需要以2为底
# [0.25, 0.25, 0.25, 0.25] * [2., 2., 2., 2.] = [0.5, 0.5, 0.5, 0.5]
a = a * tf.math.log(a)/tf.math.log(2.)

In [10]:
a

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([-0.5, -0.5, -0.5, -0.5], dtype=float32)>

In [11]:
a = tf.fill([4], 0.25)

In [12]:
-tf.reduce_sum(a*tf.math.log(a)/tf.math.log(2.))

<tf.Tensor: shape=(), dtype=float32, numpy=2.0>

In [13]:
a = tf.constant([0.1, 0.1, 0.1, 0.7])

In [14]:
-tf.reduce_sum(a*tf.math.log(a)/tf.math.log(2.))

<tf.Tensor: shape=(), dtype=float32, numpy=1.3567797>

In [15]:
# 数值越不稳定时,熵越小
a = tf.constant([0.01, 0.01, 0.01, 0.97])

In [16]:
-tf.reduce_sum(a*tf.math.log(a)/tf.math.log(2.))

<tf.Tensor: shape=(), dtype=float32, numpy=0.24194068>

In [17]:
# H(p,q) = -求和p(x)logq(x)
# H(p,q) = H(p) + DKL(p|q)
# ℎ(𝑝:0,1,0) = −1𝑙𝑜𝑔1 = 0
# 𝐻([0,1,0],[𝑝0,𝑝1,𝑝2]) = 0 + 𝐷𝐾𝐿 (𝑝|𝑞) = −𝟏𝒍𝒐𝒈𝒒𝟏
# H(label,out) = −𝟏𝒍𝒐𝒈outer
# 当outer的概率越大时,交叉熵越小,相当于预测为正确的概率也越高
# 所以使用交叉熵作为loss时,就是最小化cross_entropy

In [18]:
tf.losses.categorical_crossentropy([0,1,0,0],[0.25,0.25,0.25,0.25])

<tf.Tensor: shape=(), dtype=float32, numpy=1.3862944>

In [19]:
tf.losses.categorical_crossentropy([0,1,0,0],[0.1,0.1,0.8,0.1])

<tf.Tensor: shape=(), dtype=float32, numpy=2.3978953>

In [20]:
tf.losses.categorical_crossentropy([0,1,0,0],[0.1,0.7,0.1,0.1])

<tf.Tensor: shape=(), dtype=float32, numpy=0.35667497>

In [21]:
tf.losses.categorical_crossentropy([0,1,0,0],[0.01,0.97,0.01,0.01])

<tf.Tensor: shape=(), dtype=float32, numpy=0.030459179>

In [22]:
criteon = tf.losses.BinaryCrossentropy()

In [23]:
criteon([0,1,0,0],[0.1,0.7,0.1,0.1])

<tf.Tensor: shape=(), dtype=float32, numpy=0.168189>

In [24]:
criteon([0,1],[0.9,0.1])

<tf.Tensor: shape=(), dtype=float32, numpy=2.3025842>

In [25]:
tf.losses.BinaryCrossentropy()([1],[0.1])

<tf.Tensor: shape=(), dtype=float32, numpy=2.3025842>

In [26]:
tf.losses.binary_crossentropy([1],[0.1])

<tf.Tensor: shape=(), dtype=float32, numpy=2.3025842>

In [27]:
# Numerical Stability
x = tf.random.normal([1,784])
w = tf.random.normal([784,2])
b = tf.zeros([2])

In [28]:
# logits指的是没经过任何处理前的输出
logits = x@w+b
logits

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[33.664467 , -6.3358717]], dtype=float32)>

In [29]:
prob = tf.math.softmax(logits,axis=1)

In [30]:
prob

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[1.0000000e+00, 4.2469124e-18]], dtype=float32)>

In [31]:
tf.losses.categorical_crossentropy([[0,1]],prob)

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([16.118095], dtype=float32)>

In [32]:
# from_logits使得数值变稳定此参数代表会对数据进行softmax + cross_entropy  ********比上面的方法好的原因在于数值会稳定
tf.losses.categorical_crossentropy([[0,1]],logits,from_logits=True)

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([40.00034], dtype=float32)>